# write results to gsheet

In [3]:
import argparse
from contextlib import ExitStack
from datetime import datetime
import logging
import math
import numpy as np
import os
import pandas as pd
import pickle
import sys
import time

from fireworks import LaunchPad
from jobflow import JobStore
from jobflow.managers.fireworks import flow_to_workflow
from maggma.stores.mongolike import MongoStore
from NanoParticleTools.flows.flows import get_npmc_flow
from NanoParticleTools.inputs.nanoparticle import SphericalConstraint
import uuid

import torch
from torch.quasirandom import SobolEngine

import gpytorch
import gpytorch.settings as gpts
import pykeops
from botorch.acquisition.monte_carlo import qExpectedImprovement
from botorch.acquisition import UpperConfidenceBound
from botorch.fit import fit_gpytorch_model
from botorch.generation import MaxPosteriorSampling
from botorch.models import SingleTaskGP, ModelListGP
from botorch.optim import optimize_acqf
from botorch.test_functions import Hartmann
from botorch.utils.transforms import unnormalize
from gpytorch.constraints import Interval
from gpytorch.distributions import MultivariateNormal
from gpytorch.kernels import MaternKernel, RFFKernel, ScaleKernel
from gpytorch.kernels.keops import MaternKernel as KMaternKernel
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.mlls.exact_marginal_log_likelihood import ExactMarginalLogLikelihood

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.double
pykeops.test_torch_bindings()

from common import seed_generator
from common import utils
from common import configs
from common import gcloud_utils


/opt/anaconda3/lib/python3.9/site-packages/maggma-0.44.1-py3.9.egg/maggma/utils.py:20: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
<stdin>:1:10: fatal error: 'omp.h' file not found
#include <omp.h>
         ^~~~~~~
1 error generated.


[KeOps] Warning : omp.h header is not in the path, disabling OpenMP.
[KeOps] Warning : Cuda libraries were not detected on the system ; using cpu only mode
pyKeOps with torch bindings is working!


In [4]:


def get_results(all_done, fw_ids, DATA_DEST, from_cloud=True):
    '''get finished simulation result and log it (averaged) to log file'''
    DATA_STORE.connect()
    
    if from_cloud:
        # read data log from google sheet
        worksheet = gcloud_utils.get_ws_gspread(GSPREAD_CRED, GSPREAD_NAME)
    else:
        # read data log from file
        log = pd.read_csv(DATA_DEST)
    
    df = pd.DataFrame()
    for i, (done, fw_id) in enumerate(zip(all_done, fw_ids)):
        if done:
            uuid = get_job_uuid(fw_id)
            docs = DATA_STORE.query({'job_uuid': uuid})
            docs = list(docs)
            
            for j, doc in enumerate(docs):
                data = {}
                data['yb_1'] = 0
                data['er_1'] = 0
                data['yb_2'] = 0
                data['er_2'] = 0
                for dopant in doc['data']['input']['dopant_specifications']:
                    if dopant[0] == 0 and dopant[2] == 'Yb':
                        data['yb_1'] = dopant[1]
                    if dopant[0] == 0 and dopant[2] == 'Er':
                        data['er_1'] = dopant[1]
                    if dopant[0] == 1 and dopant[2] == 'Yb':
                        data['yb_2'] = dopant[1]
                    if dopant[0] == 1 and dopant[2] == 'Er':
                        data['er_2'] = dopant[1]

                data['radius'] = doc['data']['input']['constraints'][0]['radius']

                for spec, spec_range in RANGES.items():
                    data[spec] = utils.get_int(doc, spec_range)

                data['qe'] = utils.get_qe(doc, RANGES['TOTAL'], RANGES['ABSORPTION'])
                
                df = df.append(data, ignore_index=True)
                print(f"job {fw_id} result {j} collected.")
        else:
            print(f"job {fw_id} results abandoned.")
                
        if (i + 1) % 4 ==0:
            if df.empty:
                print("all jobs in this loop failed. check configuration.")
                # restart = input("Do want to continue loop? Press \"y\" to continue or press any key to end ")
                # if restart != "y" and restart != "Y":
                #     print("shutting down")
                #     sys.exit()
                continue
                    
            if from_cloud:
                # save data to google sheet
                stdout = worksheet.append_row(list(df.mean().round(6).values))
                if stdout['updates']['updatedRows']:
                    print("successfully updated candidate to google sheet!")
            else:
                # save data to csv file       
                log = log.append(df.mean().round(6), ignore_index=True)
                log.to_csv(DATA_DEST, index=False)
                log = pd.read_csv(DATA_DEST)
            # reset df
            df = pd.DataFrame()
    
    if from_cloud:
        log = gcloud_utils.get_df_gspread(GSPREAD_CRED, GSPREAD_NAME)
    print("all loop results sucessfully appended!")
    print("current total number of results:", len(log))
    print("current best UV:", log["UV"].max())
    print("current best structure:")
    print(log.iloc[log["UV"].idxmax()-1, :5].to_string(index=True))
    return log

def get_job_uuid(fw_id):
    DOCS_STORE.connect()
    
    fws = DOCS_STORE.query({'metadata.fw_id': fw_id})
    fws = list(fws)
    if len(fws) != 1:
        raise RuntimeError(f'found duplicated fw_id: {fw_id}')
    
    return fws[0]['uuid']

In [5]:
# credential
LP_FILE = '/Users/xiaojing/my_launchpad.yaml'
LP = LaunchPad.from_file(LP_FILE)

DOCS_STORE = MongoStore.from_launchpad_file(LP_FILE, 'test_fws_npmc')
DATA_STORE = MongoStore.from_launchpad_file(LP_FILE, 'test_docs_npmc')
LAUNCH_STORE = MongoStore.from_launchpad_file(LP_FILE, 'launches')
FWS_STORE = MongoStore.from_launchpad_file(LP_FILE, 'fireworks')

DATETIME_FORMAT = "%Y-%m-%dT%H:%M:%S.%f"

RANGES = {'UV': [300, 450],
          'VIS': [400, 700],
          'BLUE': [450, 500],
          'GREEN': [500, 590],
          'RED': [610, 700],
          'TOTAL': [200, 900],
          'ABSORPTION': [950, 1030],
         }
#all_done = [True]*4
all_done = [True, False, True, True]
fw_ids = [7375,7376,7377,7378]
DATA_DEST = "../saved_data/UV_log_shuffled_10initial_test1_grid_beta=10000.csv"
get_results(all_done, fw_ids, DATA_DEST, from_cloud=False)

job 7375 result 0 collected.
job 7375 result 1 collected.
job 7375 result 2 collected.
job 7375 result 3 collected.
job 7376 results abandoned.
job 7377 result 0 collected.
job 7377 result 1 collected.
job 7377 result 2 collected.
job 7377 result 3 collected.
job 7378 result 0 collected.
job 7378 result 1 collected.
job 7378 result 2 collected.
job 7378 result 3 collected.
all loop results sucessfully appended!
current total number of results: 24
current best UV: 7268.749098
current best structure:
yb_1       0.724286
er_1       0.055714
yb_2       0.045000
er_2       0.015000
radius    27.000000


,yb_1,er_1,yb_2,er_2,radius,UV,VIS,BLUE,GREEN,RED,TOTAL,ABSORPTION,qe
0,0.358187,0.111239,0.039788,0.370960,17.019057,37.499716,487.496133,31.249952,281.247784,162.498589,643.745197,3281.226414,0.000780
1,0.090912,0.164104,0.143426,0.266589,29.774363,24.999906,393.747768,18.749860,168.749335,199.998631,499.997404,2956.232949,0.000574
2,0.106143,0.290295,0.000000,0.000000,31.156530,68.749461,468.746097,12.499864,218.748058,224.998244,624.994693,2512.475660,0.000896
3,0.038193,0.034719,0.175456,0.212488,15.751175,12.499895,149.999232,18.749938,93.749503,37.499791,224.998868,3543.731533,0.000189
4,0.262508,0.436026,0.012062,0.810766,29.289257,43.749840,624.998175,37.499867,206.249244,381.249064,737.497787,3131.240001,0.000454
5,0.341070,0.110258,0.053051,0.020556,21.637993,1018.749107,3487.497061,231.249825,1387.498706,1643.748700,4874.995763,12537.489087,0.006349
6,0.217064,0.698471,0.043509,0.074177,26.046351,74.999821,1149.997612,43.749927,324.999321,749.998459,1368.747158,3531.242189,0.001291
7,0.765410,0.093038,0.058280,0.016624,29.078750,1241.666520,4766.666175,549.999945,1533.333189,2391.666412,6349.999319,38299.995698,0.002165
8,0.116675,0.319271,0.427934,0.173876,28.686449,18.749980,449.999089,37.499887,187.499713,218.749496,537.498845,3774.991434,0.000305
9,0.000000,0.000000,0.659883,0.173454,27.811263,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3324.993425,0.000000
